# 3. Modeling

Notebook ini mengasumsikan bahwa dataset bersih sudah disimpan sebelumnya sebagai file:

`./Dataset/UsedCarsSA_Clean.csv`

Jika belum ada, jalankan terlebih dahulu notebook notebook 02 untuk menghasilkan file tersebut.

In [74]:
# 3.1 Import library dan load dataset bersih
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Load dataset bersih
data_path_clean = '../Dataset/UsedCarsSA_Clean.csv'
real_df = pd.read_csv(data_path_clean)
df = real_df.copy()

df.head()

,Make,Type,Year,Origin,Color,Options,Engine_Size,Fuel_Type,Gear_Type,Mileage,Region,Price
0,Chrysler,C300,2018,Saudi,Black,Full,5.7,Gas,Automatic,103000,Riyadh,114000.0
1,Nissan,Sunny,2019,Saudi,Silver,Standard,1.5,Gas,Automatic,72418,Riyadh,27500.0
2,Hyundai,Elantra,2019,Saudi,Grey,Standard,1.6,Gas,Automatic,114154,Riyadh,43000.0
3,Hyundai,Elantra,2019,Saudi,Silver,Semi Full,2.0,Gas,Automatic,41912,Riyadh,59500.0
4,Honda,Accord,2018,Saudi,Navy,Full,1.5,Gas,Automatic,39000,Riyadh,72000.0


## 3.2 Menyiapkan Fitur dan Target

In [75]:
# Definisikan target dan fitur
target = 'Price'
X = df.drop(columns=[target])
y = df[target]

In [76]:
y.head()

0    114000.0
1     27500.0
2     43000.0
3     59500.0
4     72000.0
Name: Price, dtype: float64

Pada tahap ini, kolom **Price** ditetapkan sebagai **target (y)** karena merepresentasikan harga mobil yang ingin diprediksi, sedangkan **fitur (X)** terdiri dari seluruh kolom selain Price yang menggambarkan karakteristik kendaraan. Fitur tersebut mencakup atribut **kategorikal** seperti *Make, Type, Origin, Color, Options, Fuel_Type, Gear_Type,* dan *Region*, serta atribut **numerik** seperti *Year, Engine_Size,* dan *Mileage*. Kombinasi fitur-fitur ini diharapkan mampu menangkap faktor utama yang memengaruhi harga mobil bekas, mulai dari spesifikasi teknis, usia kendaraan, hingga preferensi pasar berdasarkan lokasi.

## 3.3 Train–Test Split

In [77]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

X_train.shape, X_test.shape

((3824, 11), (957, 11))

Dataset dibagi menjadi **data latih (training)** dan **data uji (testing)** dengan proporsi **80% : 20%** menggunakan `train_test_split`, di mana `random_state=42` memastikan hasil pembagian konsisten dan dapat direproduksi. Data latih digunakan untuk membangun dan melatih model, sementara data uji disisihkan untuk mengevaluasi performa model pada data yang belum pernah dilihat sebelumnya, sehingga memberikan gambaran kemampuan generalisasi model.


## 3.4 Menentukan Kolom Numerik dan Kategorikal

In [78]:
numeric_features = []
for col in ['Year', 'Engine_Size', 'Mileage', 'Car_Age']:
    if col in X_train.columns:
        numeric_features.append(col)

categorical_features = [col for col in X_train.columns if col not in numeric_features]

numeric_features, categorical_features[:10]  # tampilkan contoh beberapa fitur kategorikal

(['Year', 'Engine_Size', 'Mileage'],
 ['Make',
  'Type',
  'Origin',
  'Color',
  'Options',
  'Fuel_Type',
  'Gear_Type',
  'Region'])

Langkah ini bertujuan untuk **memisahkan fitur numerik dan kategorikal** secara eksplisit sebelum proses preprocessing. Dari hasil di atas, fitur **numerik** yang teridentifikasi adalah **Year, Engine_Size,** dan **Mileage** (sementara **Car_Age** tidak muncul karena kolom tersebut tidak tersedia di `X_train`), sedangkan fitur **kategorikal** mencakup atribut deskriptif kendaraan seperti **Make, Type, Origin, Color, Options, Fuel_Type, Gear_Type,** dan **Region**. Pemisahan ini penting karena masing-masing tipe fitur memerlukan perlakuan preprocessing yang berbeda—numerik akan diimputasi dan diskalakan, sedangkan kategorikal akan diimputasi dan diubah menjadi representasi numerik melalui one-hot encoding—sehingga pipeline modeling menjadi lebih terstruktur dan robust.


## 3.5 Pipeline Preprocessing

- Fitur numerik: imputasi median lalu scaling standar.
- Fitur kategorikal: imputasi modus lalu one-hot encoding.

Kita gunakan `ColumnTransformer` untuk menggabungkan dua jenis preprocessing ini dalam satu pipeline.

In [79]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

Pipeline preprocessing ini memisahkan perlakuan fitur **numerik** dan **kategorikal** secara sistematis: fitur numerik diimputasi menggunakan **median** untuk mengurangi pengaruh outlier lalu diskalakan dengan **RobustScaler** agar lebih stabil terhadap distribusi yang skewed, sedangkan fitur kategorikal diimputasi dengan nilai **modus** dan dikonversi menjadi representasi numerik menggunakan **One-Hot Encoding** dengan `handle_unknown='ignore'` agar aman terhadap kategori baru pada data uji. Seluruh proses digabungkan dalam `ColumnTransformer` sehingga preprocessing berlangsung konsisten, terintegrasi langsung dengan pipeline model, dan meminimalkan risiko data leakage.

## 3.6 Fungsi Evaluasi Model

In [80]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    nonzero = y_true != 0
    return np.mean(np.abs((y_true[nonzero] - y_pred[nonzero]) / y_true[nonzero])) * 100


def evaluate_regression(model, X_train, y_train, X_test, y_test, name='Model'):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae  = mean_absolute_error(y_test, y_test_pred)
    test_mape = mean_absolute_percentage_error(y_test, y_test_pred)
    test_r2   = r2_score(y_test, y_test_pred)

    print(f'=== {name} ===')
    print(f'Train MAE : {train_mae:,.2f}')
    print(f'Test MAE  : {test_mae:,.2f}')
    print(f'Test MAPE : {test_mape:.2f}%')
    print(f'Test R^2  : {test_r2:.3f}')

    return {
        'name': name,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'test_mape': test_mape,
        'test_r2': test_r2
    }

Evaluasi performa model regresi dilakukan menggunakan **tiga metrik utama** yang saling melengkapi, yaitu **MAE, MAPE, dan R²**. **MAE (Mean Absolute Error)** digunakan sebagai metrik utama karena menunjukkan rata-rata selisih absolut antara harga aktual dan prediksi dalam satuan harga yang sama, sehingga mudah diinterpretasikan secara bisnis. **MAPE (Mean Absolute Percentage Error)** digunakan untuk mengukur kesalahan prediksi secara relatif dalam bentuk persentase, sehingga memungkinkan perbandingan performa model pada mobil dengan rentang harga yang berbeda. Sementara itu, **R² (coefficient of determination)** digunakan sebagai metrik pendukung untuk menggambarkan seberapa besar variasi harga mobil yang dapat dijelaskan oleh model secara keseluruhan. Pada fungsi evaluasi, MAE dihitung untuk data latih dan data uji guna memantau potensi **overfitting**, sedangkan MAPE dan R² difokuskan pada data uji untuk menilai kemampuan generalisasi model pada data yang belum pernah dilihat sebelumnya.

## 3.7 Baseline Model: Linear Regression

Sebagai baseline, kita gunakan model regresi linear sederhana yang dikombinasikan dengan pipeline preprocessing.

In [81]:
linreg_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

linreg_model.fit(X_train, y_train)
metrics_linreg = evaluate_regression(linreg_model, X_train, y_train, X_test, y_test, 'Linear Regression')
metrics_linreg

=== Linear Regression ===
Train MAE : 11,219.46
Test MAE  : 13,151.33
Test MAPE : 111.77%
Test R^2  : 0.758


{'name': 'Linear Regression',
 'train_mae': 11219.45715124949,
 'test_mae': 13151.334845472413,
 'test_mape': np.float64(111.7723720752428),
 'test_r2': 0.7582921470331874}

## 3.8 Tree-based Models: Random Forest dan Gradient Boosting

Model berbasis pohon keputusan sering memberikan performa yang baik untuk problem regresi tabular seperti ini.

Kita coba dua model:
- RandomForestRegressor
- GradientBoostingRegressor

In [82]:
# Random Forest
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ))
])

rf_model.fit(X_train, y_train)
metrics_rf = evaluate_regression(rf_model, X_train, y_train, X_test, y_test, 'Random Forest')
metrics_rf

=== Random Forest ===
Train MAE : 4,450.46
Test MAE  : 12,218.70
Test MAPE : 116.81%
Test R^2  : 0.735


{'name': 'Random Forest',
 'train_mae': 4450.45910938434,
 'test_mae': 12218.697541299696,
 'test_mape': np.float64(116.81262794446643),
 'test_r2': 0.7345694610109201}

In [83]:
# Gradient Boosting
gbr_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(
        random_state=42
    ))
])

gbr_model.fit(X_train, y_train)
metrics_gbr = evaluate_regression(gbr_model, X_train, y_train, X_test, y_test, 'Gradient Boosting')
metrics_gbr

=== Gradient Boosting ===
Train MAE : 12,648.87
Test MAE  : 14,052.28
Test MAPE : 116.68%
Test R^2  : 0.719


{'name': 'Gradient Boosting',
 'train_mae': 12648.872253773005,
 'test_mae': 14052.283848298148,
 'test_mape': np.float64(116.68405710871914),
 'test_r2': 0.7192712912813326}

## 3.9 Model Gradient Boosting Lanjut: XGBoost

XGBoost seringkali memberikan performa yang sangat baik pada data tabular.

Kita gunakan konfigurasi awal yang sederhana dan dapat dituning lebih lanjut jika diperlukan.

In [84]:
xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42,
        n_jobs=-1
    ))
])

xgb_model.fit(X_train, y_train)
metrics_xgb = evaluate_regression(xgb_model, X_train, y_train, X_test, y_test, 'XGBoost')
metrics_xgb

=== XGBoost ===
Train MAE : 7,468.42
Test MAE  : 11,358.78
Test MAPE : 108.77%
Test R^2  : 0.785


{'name': 'XGBoost',
 'train_mae': 7468.418734243225,
 'test_mae': 11358.776026360045,
 'test_mape': np.float64(108.77199758035023),
 'test_r2': 0.7851535739918256}

## 3.10 Ringkasan Perbandingan Model

        
Kita satukan hasil evaluasi semua model ke dalam satu DataFrame untuk memudahkan perbandingan.

In [85]:
results = [metrics_linreg, metrics_rf, metrics_gbr, metrics_xgb]
results_df = pd.DataFrame(results)
results_df.sort_values('test_mae')

,name,train_mae,test_mae,test_mape,test_r2
3,XGBoost,7468.418734,11358.776026,108.771998,0.785154
1,Random Forest,4450.459109,12218.697541,116.812628,0.734569
0,Linear Regression,11219.457151,13151.334845,111.772372,0.758292
2,Gradient Boosting,12648.872254,14052.283848,116.684057,0.719271


Biasanya, model dengan **test MAE** dan **MAPE** paling rendah dan **R squared** yang cukup tinggi akan dipilih sebagai kandidat model terbaik.

Pada tahap berikutnya (Deployment), kita akan menyimpan model terbaik dan menyiapkan fungsi prediksi yang siap diintegrasikan dengan aplikasi atau API.